## Install Python packages pyikarus and dune-iga

In [1]:
%pip install pyikarus==0.3.3.dev20230620062853  --no-build-isolation --verbose --upgrade
#%pip install --force-reinstall dune-iga --no-build-isolation --verbose --upgrade
#%pip install --force-reinstall --no-deps git+git://github.com/rath3t/dune-iga/tree@main
#%pip install scikit-build
%pip install dune-iga==0.1.8.dev20230618204354 --no-build-isolation --verbose --upgrade

Using pip 23.1.2 from /dune/dune-common/build-cmake/dune-env/lib/python3.10/site-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 924.2 kB/s eta 0:00:0000:0100:01
  Preparing metadata (pyproject.toml) ...   Running command Preparing metadata (pyproject.toml)
  Note: dune-common will get version requirement (<= 2.9.0) in pyproject.toml as stated in Python-Requires.
  Note: dune-grid will get version requirement (<= 2.9.0) in pyproject.toml as stated in Python-Requires.
  Note: dune-geometry will get version requirement (<= 2.9.0) in pyproject.toml as stated in Python-Requires.
  Note: dune-istl will get version requirement (<= 2.9.0) in pyproject.toml as stated in Python-Requires.
  running dist_info
  creating /tmp/pip-modern-metadata-pc8mtmyr/pyikarus.egg-info
  writing /tmp/pip-modern-metadata-pc8mtmyr/pyikarus.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-modern-metadata-pc8mtmyr/pyikarus.egg-info/dependency_links.txt
  writing requireme

### Import packages

In [2]:
import matplotlib
import dune.grid
import dune.functions
import ikarus as iks
import ikarus.finite_elements
import ikarus.utils
import ikarus.assembler
import ikarus.dirichletValues
import numpy as np
import scipy as sp
from dune.vtk import  vtkWriter
from dune.iga import reader as readeriga
from dune.iga.basis import defaultGlobalBasis , Nurbs
from dune.iga import IGAGrid

DUNE-INFO: Registered external module ikarus


ModuleNotFoundError: No module named 'ikarus.finite_elements'

## Create grid

In [ ]:
reader = (readeriga.json, "element_trim.ibra")
gridView = IGAGrid(reader, dimgrid=2, dimworld=2)
gridView.hierarchicalGrid.globalRefine(5)

### Alternative grid

In [ ]:
#lowerLeft = [-1,-1]
##upperRight = [1,1]
#elements = [3,3]

#gridView = dune.grid.structuredGrid(lowerLeft,upperRight,elements)
#gridView.hierarchicalGrid.globalRefine(0)
#gridView.plot()

In [ ]:
%apt-get install -qq xvfb
%pip install pyvista panel -q

import os
os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

import panel as pn
pn.extension('vtk')

### Write grid to file

In [ ]:
vtkWriter = gridView.trimmedVtkWriter()
vtkWriter.write(name="TestGrid")

### Draw grid

In [ ]:
import pyvista as pv
pv.set_jupyter_backend('client')
mesh = pv.read("TestGrid.vtu")
#mesh.plot()

### Add Lagrangian basis

In [ ]:
from dune.iga.basis import Nurbs
from basishelper import globalBasis
import basishelper
import importlib
importlib.reload(basishelper)
basis = globalBasis(gridView, dune.functions.Power(Nurbs(),2))

print('We have {} dofs.'.format(len(basis.flat())))
print('We have {} vertices.'.format(gridView.size(2)))
print('We have {} elements.'.format(gridView.size(0)))

### Init load factor and displacement vector

In [ ]:
d = np.zeros(len(basis.flat()))
lambdaLoad = iks.ValueWrapper(.1)

### Create finite element requirements

In [ ]:
req = iks.FErequirements()
req.addAffordance(iks.ScalarAffordances.mechanicalPotentialEnergy)
req.addAffordance(iks.VectorAffordances.forces)
req.addAffordance(iks.MatrixAffordances.stiffness)

req.insertParameter(iks.FEParameter.loadfactor,lambdaLoad)
req.insertGlobalSolution(iks.FESolutions.displacement,d)

### Define volume load and boundary loads

In [ ]:
def volumeLoad(x,lambdaVal) :
    return np.array([lambdaVal*x[0]*2*0, 2*lambdaVal*x[1]*0])

def neumannLoad(x,lambdaVal) :
    return np.array([lambdaVal*100, lambdaVal])

indexSet = gridView.indexSet
neumannVertices = np.zeros(len(basis.flat()), dtype=bool)
#basis.interpolate(neumannVertices, lambda x :  True  if x[1]==1 else False)
#print(help(gridView))
#print(help(gridView.vertices))
for element in gridView.elements:
    for vertex in element.vertices:
        neumannVertices[indexSet.index(vertex)]= True  if vertex.geometry.center[0]==1 else False
from dune.iga import boundaryPatch
boundaryPatch = boundaryPatch(gridView,neumannVertices)

### Create vector of finite elements

In [ ]:
fes = []
for e in gridView.elements:
    fes.append(iks.finite_elements.linearElasticElement(basis,e,1000,0.2,volumeLoad,boundaryPatch,neumannLoad))


### print forces and stiffness of first element

In [ ]:
forces = np.zeros(18)
stiffness = np.zeros((18,18))
fes[1].calculateVector(req,forces)
fes[1].calculateMatrix(req,stiffness)
np.set_printoptions(precision=3)
print('Forces:\n {}'.format(forces))
print('Stiffness:\n {}'.format(stiffness))
print('Eigenvalues: ',np.real(sp.linalg.eigvals(stiffness)))

### Create Dirichlet boundary conditions

In [ ]:
dirichletValues = iks.dirichletValues(basis.flat()) 

def fixTopEdge(vec,localIndex,localView,intersection):
    if (intersection.geometry.center[1]> 1- 1e-8):
        vec[localView.index(localIndex)]= True

dirichletValues.fixBoundaryDOFsUsingLocalViewAndIntersection(fixTopEdge)

### Create assembler

In [ ]:
assemblerSparse = iks.assembler.sparseFlatAssembler(fes,dirichletValues)

Msparse = assemblerSparse.getMatrix(req)
forces = assemblerSparse.getVector(req)


### Solve for displacements and write to paraview

In [ ]:
d = sp.sparse.linalg.spsolve(Msparse,-forces)
dispFunc = basis.flat().asFunction(d)
externalForcesFunc= basis.flat().asFunction(forces)

In [ ]:
vtkWriter= gridView.trimmedVtkWriter()
vtkWriter.addPointData(dispFunc,"displacements")
vtkWriter.addPointData(externalForcesFunc,"externalForces")
vtkWriter.write(name="TestGridwithData")

### Plot here using matplot lib

In [ ]:
import dune.plotting
dune.plotting.plot(solution=dispFunc,gridLines=None)

In [ ]:
from dune.common.hashit import hashIt
from dune.generator.generator import SimpleGenerator

def ControlPointNetTest(controlPoints):
    generator = SimpleGenerator("MultiDimensionNet", "Dune::Python")

    try:
        controlPointType= controlPoints[0][0][0].cppTypeName
    except:
        try:
            controlPointType= controlPoints[0][0].cppTypeName
        except:
            try:
                controlPointType= controlPoints[0].cppTypeName
            except:
                raise Exception("Controlpoint type not deducable from list")

    element_type = f"Dune::IGA::MultiDimensionNet<{len(controlPoints)},{controlPointType}>"

    includes = []
    includes += ["dune/python/iga/nurbspatchdata.hh"]
    moduleName = "NurbsPatchData_" + hashIt(element_type)
    module = generator.load(
        includes=includes, typeName=element_type, moduleName=moduleName
    )

    return module.MultiDimensionNet(controlPoints)


In [ ]:
from dune.iga import (
    makeSurfaceOfRevolution,
    ControlPoint,
    NurbsPatchData,
    ControlPointNet)

L = 10.0

cp = ControlPoint((0, 0, 0), 1)
cp2 = ControlPoint((0, L/3.0, L/3.0), 1)
cp3 = ControlPoint((0, 1*L/3.0, 2*L/3.0), 1)
cp4 = ControlPoint((0, L, L), 1)

netC = (((cp, cp2, cp3, cp4)))
net = ControlPointNet(netC)
nurbsPatchData = NurbsPatchData(((0, 0, 1.0/3.0, 2.0/3.0, 1, 1)), net, (1))

maxIter = 100

for i in range(0,maxIter):

    nurbsPatchDataArc = makeSurfaceOfRevolution(nurbsPatchData, (0, 0, L*i/maxIter), (1, 0.5, 0), 360.0)
    gridView = IGAGrid(nurbsPatchDataArc)
    vtkWriter = gridView.vtkWriter(subsampling=4)
    vtkWriter.write(name=f"Torus_{i}")